In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from keras.models import *
from keras.layers import *
from keras.datasets import mnist
import os

In [ ]:

OUT_DIR = './CNN_out'
img_shape = (28, 28, 1)
epochs = 100000
batch_size = 128
noise = 100
sample_interval = 100

In [ ]:

(x_train, _), (_, _) = mnist.load_data()
print(x_train.shape)

x_train = x_train / 127.5 - 1
x_train = np.expand_dims(x_train, axis=3)         # expand_dims : 차원 확장
print(x_train.shape)

11490434/11490434 [==============================] - 2s 0us/step
(60000, 28, 28)
(60000, 28, 28, 1)


In [ ]:
generator = Sequential()
generator.add(Dense(256*7*7, input_dim=noise))
generator.add(Reshape((7, 7, 256)))
generator.add(Conv2DTranspose(128, kernel_size = 3, strides = 2, padding = 'same'))
generator.add(BatchNormalization())
generator.add(LeakyReLU(alpha=0.01))
generator.add(Conv2DTranspose(64, kernel_size = 3, strides = 1, padding = 'same'))
generator.add(BatchNormalization())
generator.add(LeakyReLU(alpha=0.01))
generator.add(Conv2DTranspose(1, kernel_size = 3, strides = 2, padding = 'same'))
generator.add(Activation('tanh'))
generator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12544)             1266944   
                                                                 
 reshape (Reshape)           (None, 7, 7, 256)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 14, 14, 128)      295040    
 nspose)                                                         
                                                                 
 batch_normalization (BatchN  (None, 14, 14, 128)      512       
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 14, 14, 128)       0         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 14, 14, 64)       7

In [ ]:
discriminator = Sequential()
discriminator.add(Conv2D(32, kernel_size = 3, strides = 2, padding = 'same', input_shape=img_shape))   # strides를 쓸 경우, MaxPoolD를 굳이 쓰지 않아도 된다.
discriminator.add(LeakyReLU(alpha = 0.01))

discriminator.add(Conv2D(64, kernel_size = 3, strides = 2, padding = 'same'))
discriminator.add(LeakyReLU(alpha = 0.01))
discriminator.add(Conv2D(128, kernel_size = 3, strides = 2, padding = 'same'))
discriminator.add(LeakyReLU(alpha = 0.01))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 32)        320       
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 14, 14, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 64)          18496     
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 7, 7, 64)          0         
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 4, 4, 128)         0         
                                                                 
 flatten (Flatten)           (None, 2048)             

In [ ]:
discriminator.compile(loss='binary_crossentropy', optimizer='adam',
                      metrics=['accuracy'])
discriminator.trainable = False

gan_model = Sequential()
gan_model.add(generator)
gan_model.add(discriminator)
gan_model.summary()
gan_model.compile(loss='binary_crossentropy', optimizer='adam')

real = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 28, 28, 1)         1637121   
                                                                 
 sequential_1 (Sequential)   (None, 1)                 94721     
                                                                 
Total params: 1,731,842
Trainable params: 1,636,737
Non-trainable params: 95,105
_________________________________________________________________


In [ ]:
for epoch in range(epochs):
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    real_imgs = x_train[idx]

    z = np.random.normal(0, 1, (batch_size, noise))
    fake_imgs = generator.predict(z)

    d_hist_real = discriminator.train_on_batch(real_imgs, real)
    #.fit : epoch만큼 반복 학습 / .train_on_batch : 반복하지 않고 batch를 준 묶음만 1회 학습 후 종료
    d_hist_fake = discriminator.train_on_batch(fake_imgs, fake)

    d_loss, d_acc = np.add(d_hist_fake, d_hist_real) * 0.5

    discriminator.trainable = False

    if epoch % 2 ==0:
        z = np.random.normal(0, 1, (batch_size, noise))
        gan_hist = gan_model.train_on_batch(z, real)

    if epoch % sample_interval == 0:
        print('%d, [D_loss: %f, acc.: %.2f%%], [G_loss: %f]'%(
            epoch, d_loss, d_acc, gan_hist))
        row = col = 4
        z = np.random.normal(0, 1, (row * col, noise))
        fake_imgs = generator.predict(z)
        fake_imgs = 0.5 * fake_imgs + 0.5
        _, axs = plt.subplots(row, col, figsize=(5, 5), sharey=True, sharex=True)
        cont = 0
        for i in range(row):
            for j in range(col):
                axs[i, j].imshow(fake_imgs[cont, :, :, 0], cmap='gray')
                axs[i, j].axis('off')
                cont += 1
        path = os.path.join(OUT_DIR, 'img-{}'.format(epoch+1))
        plt.savefig(path)
        plt.close()


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
4/4 [==============================] - 0s 4ms/step
14700, [D_loss: 0.271151, acc.: 0.89%], [G_loss: 3.209832]
4/4 [==============================] - 0s 4ms/step
14800, [D_loss: 0.234063, acc.: 0.90%], [G_loss: 3.279039]
4/4 [==============================] - 0s 7ms/step
14900, [D_loss: 0.158997, acc.: 0.93%], [G_loss: 3.705842]
4/4 [==============================] - 0s 4ms/step
15000, [D_loss: 0.169419, acc.: 0.93%], [G_loss: 2.949425]
4/4 [==============================] - 0s 3ms/step
15100, [D_loss: 0.235171, acc.: 0.91%], [G_loss: 3.505072]
4/4 [==============================] - 0s 4ms/step
15200, [D_loss: 0.103754, acc.: 0.96%], [G_loss: 4.829161]
4/4 [==============================] - 0s 6ms/step
15300, [D_loss: 0.204982, acc.: 0.92%], [G_loss: 4.092808]
4/4 [==============================] - 0s 5ms/step
15400, [D_loss: 0.221824, acc.: 0.90%], [G_loss: 4.016834]
4/4 [==============================] - 0s 6ms/step
15500, [D_loss: 0.136557, acc.: 0